<a href="https://colab.research.google.com/github/SakayanagiTOYOTA/forLLM_edu/blob/main/pdf2rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PDFファイルからRAGを作成

参考URL https://qiita.com/tatsuki-tsuchiyama/items/179c1ded40b9c54de09a

PDFを読込み、RAGとして保存する。本スクリプト実行後にloadrag.pyを実行する事

In [ ]:
# AzureOpenAIの場合
import os
os.environ["AZURE_OPENAI_API_KEY"] = "xxxxxxx"
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://aoai-aoai04-japaneast.openai.azure.com/'
DEPLOYMENT_GPT4o='gpt-4o-2024-05-13'
API_VERSION_GPT4o='2024-02-15-preview'
DEPLOYMENT_EMBEDDING_ADA='text-embedding-ada-002-2'
API_VERSION_EMBEDDING_ADA='2023-05-15'
DEPLOYMENT_EMBEDDING_LARGE='text-embedding-3-large-1'
API_VERSION_EMBEDDING_LARGE='2023-05-15'

MODEL_TYPE='AzureOpenAI'

In [1]:
# AWS Bedrockの場合
import os
os.environ['AWS_ACCESS_KEY_ID'] = 'xxxxxx'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'xxxxxxxx'
os.environ['AWS_SESSION_TOKEN'] = 'xxxxxxx'
os.environ['AWS_DEFAULT_REGION'] = 'ap-northeast-1'

MODEL_TYPE='Bedrock'

Google Colaboの場合にライブラリインストール

In [5]:
!pip install langchain-aws pypdf  faiss-cpu

定数設定

In [3]:
CHUNK_SIZE = 1000
PDF_PATH = 'OCPP-2.0.1_part2_specification.pdf'

モデルの読込

In [6]:
if MODEL_TYPE == 'AzureOpenAI':
  from langchain.embeddings import AzureOpenAIEmbeddings
  embeddings = AzureOpenAIEmbeddings(azure_deployment=DEPLOYMENT_EMBEDDING_LARGE,
    chunk_size=CHUNK_SIZE
  )

elif MODEL_TYPE == 'Bedrock':
  from langchain_aws import BedrockEmbeddings
  embeddings=BedrockEmbeddings(model_id="amazon.titan-embed-text-v1",region_name="ap-northeast-1")


## PDF読込

Google Colaboの場合はPDF読み込む前にPDFファイルをアップロードしてください。

In [8]:
# PDFを読み込み
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader(PDF_PATH)
pages = loader.load_and_split()

ページの一部を表示

In [9]:
print(pages[1])

page_content='Table of Contents
Disclaimer . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .  1
Generic. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .  2
Version History. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .  3
1. Scope . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .  4
1.1. OCPP 2.0.1 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

## ベクトル化

In [10]:
# 空のベクトルストアを作成
from langchain.vectorstores import FAISS
dummy_text, dummyy_id = "1",1
vectorstore = FAISS.from_texts([dummy_text],embeddings, ids=[dummyy_id])
vectorstore.delete([dummyy_id])

True

In [11]:
# ページごとにベクトル化
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter( chunk_size = CHUNK_SIZE)
for page in pages:
    docs = text_splitter.split_documents([page])
    vectorstore.merge_from(FAISS.from_documents(docs,embeddings))
#    print(f"ベクトル化完了: {page.metadata["source"]} - ページ {page.metadata["page"]}")
    print(f"finish: {page.metadata['source']} - page {page.metadata['page']}")

finish: OCPP-2.0.1_part2_specification.pdf - page 0
finish: OCPP-2.0.1_part2_specification.pdf - page 1
finish: OCPP-2.0.1_part2_specification.pdf - page 1
finish: OCPP-2.0.1_part2_specification.pdf - page 1
finish: OCPP-2.0.1_part2_specification.pdf - page 2
finish: OCPP-2.0.1_part2_specification.pdf - page 2
finish: OCPP-2.0.1_part2_specification.pdf - page 2
finish: OCPP-2.0.1_part2_specification.pdf - page 3
finish: OCPP-2.0.1_part2_specification.pdf - page 3
finish: OCPP-2.0.1_part2_specification.pdf - page 3
finish: OCPP-2.0.1_part2_specification.pdf - page 3
finish: OCPP-2.0.1_part2_specification.pdf - page 4
finish: OCPP-2.0.1_part2_specification.pdf - page 4
finish: OCPP-2.0.1_part2_specification.pdf - page 4
finish: OCPP-2.0.1_part2_specification.pdf - page 5
finish: OCPP-2.0.1_part2_specification.pdf - page 5
finish: OCPP-2.0.1_part2_specification.pdf - page 5
finish: OCPP-2.0.1_part2_specification.pdf - page 5
finish: OCPP-2.0.1_part2_specification.pdf - page 6
finish: OCPP

## RAGとして保存

In [12]:
vectorstore.save_local("./vectorstore")

# RAGを用いて回答する

先ほど保存されたRAGのファイルを用いて回答するチャットアプリを作成する

モデルを作成

In [13]:
if MODEL_TYPE == 'AzureOpenAI':
  from langchain_openai import AzureChatOpenAI
  model = AzureChatOpenAI(azure_deployment=DEPLOYMENT_GPT4o, api_version=API_VERSION_GPT4o)

elif MODEL_TYPE == 'Bedrock':
  from langchain_aws import ChatBedrock
  model = ChatBedrock(model_id="anthropic.claude-3-5-sonnet-20240620-v1:0")

プロンプトを作成

In [14]:
from langchain.prompts import PromptTemplate

template = """
<bos><start_of_turn>system
次の文脈を使用して、最後の質問に答えてください。
{context}
<end_of_turn>
<start_of_turn>user
{query}
<end_of_turn><start_of_turn>model
"""

prompt = PromptTemplate(
    template=template
)

chainを作成

In [15]:
chain = prompt | model


## RAGを読込
モデル読込

In [17]:
if MODEL_TYPE == 'AzureOpenAI':
  from langchain.embeddings import AzureOpenAIEmbeddings
  embeddings = AzureOpenAIEmbeddings(
    azure_deployment=DEPLOYMENT_EMBEDDING_LARGE,
    chunk_size=CHUNK_SIZE
  )
elif MODEL_TYPE == 'Bedrock':
  from langchain_aws import BedrockEmbeddings
  embedding=BedrockEmbeddings(model_id="amazon.titan-embed-text-v1",region_name="ap-northeast-1")

ベクトルストア読込

In [18]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.load_local("./vectorstore",embeddings , allow_dangerous_deserialization=True)

質問用意

In [19]:
query = "充電器が停電時に考慮するべき点について教えてください。"

RAGから質問に関連した文章を検索

In [26]:
docs = vectorstore.similarity_search(query=query, k=5)
content = "\n".join([f"Content:\n{doc.page_content}" for doc in docs])

ヒットした文章を表示

In [27]:
print(content)

Content:
Value Description
W Watts (power). This is the TOTAL allowed charging power. If used for AC Charging, the phase current should
be calculated via: Current per phase = Power / (Line Voltage * Number of Phases). The "Line Voltage" used in
the calculation is not the measured voltage, but the set voltage for the area (hence, 230 of 110 volt). The
"Number of Phases" is the numberPhases from the ChargingSchedulePeriod. It is usually more convenient to
use this for DC charging. Note that if numberPhases in a ChargingSchedulePeriod is absent, 3 SHALL be
assumed.
A Amperes (current). The amount of Ampere per phase, not the sum of all phases. It is usually more
convenient to use this for AC charging.
3.16. ChargingStateEnumType
Enumeration
The state of the charging process.
ChargingStateEnumType is used by: transactionEvent:TransactionEventRequest.TransactionType
Value Description
Charging The contactor of the Connector is closed and energy is flowing to between EVSE and EV.
EVConnected 

RAGから検索した文章を考慮して回答

In [28]:
answer = chain.invoke({'query':query, 'context':content})
print(answer.content)

充電器が停電時に考慮すべき主なポイントは以下の通りです:

1. スタンドアロン操作:
通信が利用できない場合でも、充電器は単独で動作できるように設計されている必要があります。

2. オフライン状態の管理:
充電器はオフライン状態を適切に管理し、接続が回復したときに正しく対応する必要があります。

3. ステータス通知:
接続が回復した後、オフライン期間が設定された閾値を超えた場合は、すべてのコネクタのステータスを通知する必要があります。閾値内の場合は、状態が変化したコネクタのみ通知します。

4. 再接続の試行:
充電器は定期的に中央システムとの接続再確立を試みる必要があります。

5. 状態の維持:
予約済みや利用不可などの状態は、再起動後も維持される必要があります。

6. セルフチェック:
電源復帰後、必要に応じてハードウェアのセルフチェックを実施します。

7. ブート通知:
電源復帰時には適切なタイミングでBootNotificationRequestを送信し、システムに再接続する必要があります。

これらの点に注意することで、停電時や通信障害時でも充電器が適切に動作し、復旧後に正しく中央システムと同期できるようになります。
